<a href="https://www.kaggle.com/code/fabiancarmona/google-data-analytics-cyclistic?scriptVersionId=241304712" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Google Data Analytics Capstone Project
## Case Study: Cyclistic, bike-share company

## 1. Business task:
The director of marketing of Cyclistyc, a bike-share company in Chicago, believes that the company's future success will depend on maximizing the annual memberships. In this case there are two types of customers in Cyclistic, the casual riders and the annual members, the objective is to convert the casual riders into annual members but first the Cyclistic marketing analytics team need to understand how these two types of customers use Cyclistic bikes differently, so we can find some insights to design a marketing strategy to achieve this goal.

## 2. Data sources:
For this case study the data sources used were the 2 .csv file datasets provided by Motivate International Inc. Which is a company that is focused on the bike-sharing and provides operation and logistics support to the micromobility.

Before importing the datasets I added 2 columns to each of them in a spreadsheet.
* 1 column named ride_length with the ending time substracting the start time so I can see the duration of the ride.
* 1 column named day_of_week, in this one I get the day of the week of the start date with the function WEEKDAY() and I also use the function CHOOSE() to assign the name of the week instead of a number.

In [ ]:
# To get everything ready for the cleaning process, first we need to 
# load the packages and import the datasets that we'll need.

library(tidyverse)

# Here we assign the datasets to these 2 variables.
df1 = read_csv('../input/cyclistic-datasets/Divvy_Trips_2019_Q1.csv')
df2 = read_csv('../input/cyclistic-datasets/Divvy_Trips_2020_Q1.csv')

## 3. Cleaning process:


In [ ]:
# Then I can take a look to the first rows of our data with
# the funcion head().

head(df1)
print("----------------------------------------------------------------------------")
head(df2)

In [ ]:
# I can also take a look at the dataframes with the glimpse() function
# and with this we can see how many rows and columns they have.
glimpse(df1)
print("----------------------------------------------------------------------------")
glimpse(df2)

As we saw before the 2 dataframes have a lot of rows, so I think that continuing to use a spreadsheet wouldn't be a good option to clean the data because these are large datasets, In this case I will continue using R.

In [ ]:
# To clean the data I will use the next functions:

# First I assign the dataframe to a new variable called "clean_df1".
clean_df1 <- df1 %>%
    # Here I want to modify all the columns of the dataframe,
    # I specify that I want to apply the same function to many 
    # columns in this case all of them and I clean any extra blank space.
    mutate(across(everything(), str_trim)) %>%
    # Then I delete the NA values in the dataframe.
    drop_na() %>%
    # And with this function I make sure to delete the duplicate values.
    unique.data.frame()

In [ ]:
# Then I do the same with the "df2"
clean_df2 <- df2 %>%
    mutate(across(everything(), str_trim)) %>%
    drop_na() %>%
    unique.data.frame()

In [ ]:
# I could use the glimpse() function again to check the number of rows one more time.
# When we run this code we'll see that the number of rows decreased because of the
# values that we dropped before cleaning the data.

glimpse(clean_df1)
print("----------------------------------------------------------------------------")
glimpse(clean_df2)

## 4. Summary of analysis:


In [ ]:
# Here we can take a look at the column names of each dataset.
colnames(clean_df1)
print("----------------------------------------------------------------------------")
colnames(clean_df2)

In [ ]:
# I need to bind the rows of the datasets to make one single dataframe, for that first
# I rename the names of the columns of the first dataframe so I can give them the
# same name of the other dataframe because they have the same data and with 
# this all the information of the 2 dataframes will be in one column with the same 
# name and in only just one dataframe.
clean_df1 <- rename(clean_df1,
                   ride_id = trip_id,
                   rideable_type = bikeid,
                   started_at = start_time,
                   ended_at = end_time,
                   start_station_name = from_station_name,
                   start_station_id = from_station_id,
                   end_station_name = to_station_name,
                   end_station_id = to_station_id,
                   member_casual = usertype)

In [ ]:
# Here we take one more look at the column names and as we can see
# now the names are the same.
colnames(clean_df1)
print("----------------------------------------------------------------------------")
colnames(clean_df2)

In [ ]:
# Then I assign the 2 dataframes in the variable called "all_trips"
# Now all the rows are part of just one dataframe.
all_trips <- bind_rows(clean_df1, clean_df2)

In [ ]:
# Then we will discard the columns that didn't have the same data in the 2 dataframes
# because there will be a lot of NA values. So we assign the change to the same variable.
all_trips <- all_trips %>%
    # Here we select all the data of "all_trips" minus these column names.
    select(-c(start_lat, start_lng, end_lat, end_lng, birthyear, gender, tripduration))

In [ ]:
# A summary of "all_trips"
print("column names")
colnames(all_trips)
print("number of rows")
nrow(all_trips)
print("dimension of the dataframe (rows and columns)")
dim(all_trips)
print("first 6 rows")
head(all_trips)
print("structure of the dataframe")
str(all_trips)
print("summary of all the columns")
summary(all_trips)

In [ ]:
# Now with these we count the number of each values that contain the
# member_casual column, as we can see there 4 different names but 2 of them
# have the same meaning of the other 2, for these we need to make just one of them
table(all_trips$member_casual)

In [ ]:
# So the objective is to leave just the most updated names in these case it will
# be member and casual.
all_trips = all_trips %>%
    # I will update the member_casual column.
    mutate(member_casual = case_match(
        member_casual,
        # Change Subscriber for member.
        "Subscriber" ~ "member",
        # Change Customer for casual.
        "Customer" ~ "casual",
        # Leave all of the other texts as they are (member and casual)
        .default = member_casual))

In [ ]:
# And now we just have these to names.
table(all_trips$member_casual)

In [ ]:
safe_spot <- all_trips

In [ ]:
# Here we create new columns to separate the date.
all_trips$date <- as.Date(all_trips$started_at) # Here we get only the date without time.
all_trips$year <- format(as.Date(all_trips$date), "%Y") # The year
all_trips$month <- format(as.Date(all_trips$date), "%m") # The month
all_trips$day <- format(as.Date(all_trips$date), "%d") # The day

In [ ]:
# Here we update the ride_length column an now it will be in seconds.
all_trips$ride_length <- difftime(all_trips$ended_at, all_trips$started_at)

In [ ]:
# Change to numeric
all_trips$ride_length <- as.numeric(all_trips$ride_length)

In [ ]:
# Now it is numeric.
is.numeric(all_trips$ride_length)
is.difftime(all_trips$ride_length)

In [ ]:
# Then here I was asked to removed "bad" data, in this case it's the all the data
# from the "HQ QR" station and any ride_length that have negative values and we
# will assign this to a new variable called "all_trips_v2"
all_trips_v2 <- all_trips[!(all_trips$start_station_name == "HQ QR" | all_trips$ride_length<0),]

In [ ]:
# We can see how many rows we removed
# Before
nrow(all_trips)
cat("----------------------------------------------------------------------------")
# After
nrow(all_trips_v2)

In [ ]:
# Here we can make some calculations.
print("average")
mean(all_trips_v2$ride_length)
print("median")
median(all_trips_v2$ride_length)
print("maximum")
max(all_trips_v2$ride_length)
print("minimum")
min(all_trips_v2$ride_length)

In [ ]:
# The function summary() can do the same calculations.
summary(all_trips_v2$ride_length)

In [ ]:
# With the aggregate() function we can select the column that will apply a
# function and the column that we want to compare in these case there are
# comparisons of the ride length between casual riders and member riders with the
# functions we used before.

print("average")
aggregation1 <- aggregate(all_trips_v2$ride_length ~ all_trips_v2$member_casual, FUN = mean)
# Here's how we can assign names to the columns, otherwhise the function will use,
# the default name that we give it.
names(aggregation1) <- c("user_type", "average_ride_length")
aggregation1

print("median----------------------------------------------------------------------")
aggregate(all_trips_v2$ride_length ~ all_trips_v2$member_casual, FUN = median)
print("maximum----------------------------------------------------------------------")
aggregate(all_trips_v2$ride_length ~ all_trips_v2$member_casual, FUN = max)
print("minimum----------------------------------------------------------------------")
aggregate(all_trips_v2$ride_length ~ all_trips_v2$member_casual, FUN = min)

In [ ]:
# Before make a calculation column of the days of the week, we can order them
# like this.
all_trips_v2$day_of_week <- ordered(all_trips_v2$day_of_week, 
                                    levels=c("Sunday", "Monday", "Tuesday", "Wednesday", 
                                             "Thursday", "Friday", "Saturday"))

In [ ]:
# Now the days of the week will be in order.
aggregate(all_trips_v2$ride_length ~ all_trips_v2$member_casual + 
          all_trips_v2$day_of_week, FUN = mean)

In [ ]:
# Then we create a table with the next information and assign that table
# to the variable "ridership_data" which we'll use for vizualizations.
ridership_data <- all_trips_v2 %>%
    # Data will be grouped by the member_casual and the day_of_week columns.
    group_by(member_casual, day_of_week) %>%
    # Then we count the number of rows for each day and each user, we'll assign that
    # to "number_of_rides".
    summarise(number_of_rides = n(),
              # This will be the average duration of the rides of each day and user. 
              average_duration = mean(ride_length)) %>%
    # And then we sort the data by member_casual and day_of_week.
    arrange(member_casual, day_of_week)

In [ ]:
# We just run this variable to see the table.
ridership_data

## 5. Vizualizations:

In [ ]:
plot1 <- ridership_data %>%
    # Now with the function ggplot() we create the visualizations, in this case it's a
    # visualization of number of rides of each user each day.
    ggplot(aes(x=day_of_week, y=number_of_rides, fill=member_casual)) +
    geom_col(position = "dodge")

plot1

In [ ]:
plot2 <- ridership_data %>%
    # In this viz instead of the number of rides we are using the
    # average duration of the rides.
    ggplot(aes(x=day_of_week, y=average_duration, fill=member_casual)) +
    geom_col(position = "dodge")

plot2

In [ ]:
# We can save the graphs with ggsave()
ggsave("plot1.png", plot1)
ggsave("plot2.png", plot2)

In [ ]:
# Here we can one of the table that we make before and
# assign it to a variable
counts <- aggregate(all_trips_v2$ride_length ~ all_trips_v2$member_casual +
                   all_trips_v2$day_of_week, FUN = mean)

# We can use this function if we want to export our table in a .csv file.
# In my case I will leave it in a comment.

# write.csv(counts, file="test.csv")

## 6. Recommendations:
* The casual riders usually take more rides on the weekends we could take advange of that, maybe offering a weekend promotion of annual memberships.
  
* For the casual riders that used to ride during weekdays we could create some perks with exclusive benefits for the annual members.
  
* We could also offer a special discount for the new members to buy the annual memberships.